In [1]:
import tensorflow as tf
import time
from mtcnn import MTCNN
from tqdm import tqdm
import numpy as np
import os
import shutil
import cv2

In [84]:
def crop_images(sdir,dst_dir, min_img_size, height, width):
    # check if the dst_dir exists. If not create it. If it exist check if it contains files.
    # if it does query the user to keep or delete those files
    if os.path.isdir(dest_dir)==False: # the dst_dir does not exist create it
        os.mkdir(dst_dir)
    else:
        if (len(os.listdir(dst_dir))) >0:
            print ('the dest_dir ', dst_dir, ' contains files enter D to delete them or press ENTER to preserve them')
            ans=input(' ')
            if ans =='D' or ans =='d':
                shutil.rmtree(dst_dir)
                os.mkdir(dst_dir)
                
    detector = MTCNN() 
    start=time.time()
    flist=sorted(os.listdir(sdir))    
    no_face_list=[]   # list filepaths where no face was detected in the image
    to_small_list=[]  # list filepaths where the cropped image was to small and not saved 
    bad_img_list=[]   # list file paths that were invalid image files
    saved=0
    processed=0
    for f in tqdm(flist,  unit='files', ncols=100, leave=False): 
        processed +=1
        fpath=os.path.join(sdir,f)
        dst_fpath=os.path.join(dst_dir, f)
        try:
            img=cv2.imread(fpath)
            data=detector.detect_faces(img)            
            if len(data)>0:
                # iterate throug the faces found and determine which image has the most pixels
                biggest=0                    
                for i, faces in enumerate(data): # iterate through all the faces found                    
                    box=faces['box']  # get the box for each face                   
                    area = box[3]  * box[2]                            
                    if area>biggest:
                        biggest=area
                        bbox=box
                        ibiggest=i         
            
                    # create the cropped image of the largest face
                    bimg=img[bbox[1]: bbox[1]+bbox[3],bbox[0]: bbox[0]+ bbox[2]]                     
                    if height > 0 and width > 0:
                        print('rrrheight: ', height, '  rrrwidth: ', width)
                        h=height
                        w=width                       
                    elif height>0 and width==0:
                        w=int(height*bimg.shape[1]/bimg.shape[0])
                        h=height                    
                    elif height==0 and width >0:
                        h= int(width*bimg.shape[0]/bimg.shape[1])
                        w=width
                    else:
                        h=bimg.shape[0]
                        w=bimg.shape[1]                     
                    
                    if bimg.shape[0]>=min_img_size and bimg.shape[1]>=min_img_size:
                        img=cv2.resize(bimg, (w,h))
                        cv2.imwrite(dst_fpath, img)
                        saved +=1
                    else:
                        to_small_list.append(fpath)                
            else:
                no_face_list.append(fpath)
        except:
            bad_img_list.append(fpath)
        
    end=time.time()
    duration=end-start
    hours = duration // 3600
    minutes = (duration - (hours * 3600)) // 60
    seconds = duration - ((hours * 3600) + (minutes * 60))
    print(f'Process completed in {hours} hours {minutes} minutes {seconds:3.0f} seconds')
    print(f'{processed} files were processed, {saved} files were saved to {dst_dir}')
    return to_small_list, no_face_list, bad_img_list

In [85]:
# example of use
sdir=r'C:\Temp\Autism\images to predict' # directory where images to be cropped are stored
dst_dir=r'C:\Temp\Autism\cropped'  # directory where cropped images will be stored
min_img_size=100 # min height and width a cropped image must have or exceed in order to be saved
height=200 # height of image to be saved. If set to 0 height is calculated to preserve aspect ratio of cropped image
width=0    # width of image to be saved, if set to 0 width of image to be saved is calculated to preserve aspect ratio of cropped image
# if width=0 and height=0 the cropped image is not resized
to_small_list, no_face_list, bad_img_list = crop_images(sdir,dst_dir, min_img_size, height, width)
print('to_small_list\n',to_small_list)
print ('\nno_face_detected list\n',no_face_list)
print ('\nbad_img_list\n',bad_img_list)

the dest_dir  C:\Temp\Autism\cropped  contains files enter D to delete them or press ENTER to preserve them
 d


Process completed in 0.0 hours 0.0 minutes   3 seconds
4 files were processed, 2 files were saved to C:\Temp\Autism\cropped
to_small_list
 ['C:\\Temp\\Autism\\images to predict\\3.jpg']

no_face_detected list
 ['C:\\Temp\\Autism\\images to predict\\wabc.jpg']

bad_img_list
 []
